In [148]:
from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances
import pandas as pd
get_ipython().magic('matplotlib')

df=pd.read_csv('C:/Scenarios about Rare Diseases_2017_08_09.csv',sep=";",encoding="latin1")
to_del = ['Name']
print (to_del)
filtered_cols = [c for c in df.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df[filtered_cols]

print ("df_2",df_2.shape)
print(df_2.columns)

Using matplotlib backend: Qt5Agg
['Name']
df_2 (25, 24)
Index(['Age', 'Country', 'Disease', 'Age of Diagnosis', 'Treatment',
       'InterestedIn', 'FacilitatorTechCommunication',
       'RemunerativeEmployment', 'BarrierHealthProfessionals',
       'EmotionalFunctions', 'Non-RemunerativeEmployment', 'Consciousness',
       'Vomiting', 'Respiratory functions', 'Skin functions',
       'Hearing and Vestibular Function', 'Financial Assets', 'Health systems',
       'Higher Education', 'Cognitive Functions', 'Pain in head and neck',
       'Sports', 'Arts and culture', 'Walking'],
      dtype='object')


In [149]:
print(df_2.columns)

Index(['Age', 'Country', 'Disease', 'Age of Diagnosis', 'Treatment',
       'InterestedIn', 'FacilitatorTechCommunication',
       'RemunerativeEmployment', 'BarrierHealthProfessionals',
       'EmotionalFunctions', 'Non-RemunerativeEmployment', 'Consciousness',
       'Vomiting', 'Respiratory functions', 'Skin functions',
       'Hearing and Vestibular Function', 'Financial Assets', 'Health systems',
       'Higher Education', 'Cognitive Functions', 'Pain in head and neck',
       'Sports', 'Arts and culture', 'Walking'],
      dtype='object')


In [150]:
print(type(df_2))
df_3 = pd.get_dummies( data=df_2,columns=['Country','Disease','Treatment','InterestedIn'])
print(df_3.columns)
for i in range(0,df_3.shape[0]):
    if (df_3.loc[i,'Age of Diagnosis']=='None'):
        df_3.loc[i,'Age of Diagnosis']="NaN"
df_3['Age']=df_3['Age'].astype(float)
df_3['Age of Diagnosis']=df_3['Age of Diagnosis'].astype(float)

<class 'pandas.core.frame.DataFrame'>
Index(['Age', 'Age of Diagnosis', 'FacilitatorTechCommunication',
       'RemunerativeEmployment', 'BarrierHealthProfessionals',
       'EmotionalFunctions', 'Non-RemunerativeEmployment', 'Consciousness',
       'Vomiting', 'Respiratory functions', 'Skin functions',
       'Hearing and Vestibular Function', 'Financial Assets', 'Health systems',
       'Higher Education', 'Cognitive Functions', 'Pain in head and neck',
       'Sports', 'Arts and culture', 'Walking', 'Country_France',
       'Country_Iran', 'Country_Sierra Leone', 'Country_Spain',
       'Disease_Acute Lymphoblastic Leukaemia', 'Disease_Algoneurodystrophy',
       'Disease_Arthropathy in hypersensitivity reactions classified elsewhere',
       'Disease_Benign intracranial hypertension',
       'Disease_Cerebral arteritis, not elsewhere classified',
       'Disease_Congenital malformation short stature',
       'Disease_Congenital malformation syndromes predominantly affecting facial 

In [151]:
#Fill na
df_4 = df_3.fillna(value=np.mean(df_3,axis=0),inplace=False,axis=0).values
scaler = preprocessing.MinMaxScaler().fit(df_4)
data = scaler.transform(df_4)
print ("data",data.shape)
thrd = 0.8
total = 0
pca = PCA().fit(data)
reduced_data = pca.transform(data)
for pca_comps,r in enumerate(pca.explained_variance_ratio_):
    if total > thrd:
        break
    total += r
print ("Num pca_comps per >", thrd,"ratio:", pca_comps, total)
print ("Explained variance first 2 components",pca.explained_variance_ratio_[0]+pca.explained_variance_ratio_[1])
print (pca.n_components_)
print (np.sum(pca.explained_variance_ratio_[:2]))

print ("PCA+K-means:", pca_comps)

data (25, 51)
Num pca_comps per > 0.8 ratio: 13 0.803982782505
Explained variance first 2 components 0.321772935951
51
0.321772935951
PCA+K-means: 13


In [152]:
from sklearn.metrics import silhouette_samples, silhouette_score
for k in range(2,11):
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
    cluster_labels = kmeans.fit_predict(reduced_data[:,:pca_comps])
    #cluster_labels=kmeans.fit(reduced_data[:,:2])
    silhouette_avg = silhouette_score(reduced_data[:,:pca_comps], cluster_labels)
    #silhouette_avg = silhouette_score(reduced_data[:,:2], cluster_labels)
    print("For n_clusters =", k, "The average silhouette_score is :", silhouette_avg)
    calinski_harabaz_score_avg = metrics.calinski_harabaz_score(reduced_data[:,:pca_comps], cluster_labels)
    #calinski_harabaz_score_avg = metrics.calinski_harabaz_score(reduced_data[:,:2], cluster_labels)
    print("For n_clusters =", k," the average metrics.calinski_harabaz_score is :", calinski_harabaz_score_avg)

For n_clusters = 2 The average silhouette_score is : 0.24243515601
For n_clusters = 2  the average metrics.calinski_harabaz_score is : 6.3768639144
For n_clusters = 3 The average silhouette_score is : 0.2898603087
For n_clusters = 3  the average metrics.calinski_harabaz_score is : 6.52424963582
For n_clusters = 4 The average silhouette_score is : 0.323897848518
For n_clusters = 4  the average metrics.calinski_harabaz_score is : 5.94745653135
For n_clusters = 5 The average silhouette_score is : 0.249692240674
For n_clusters = 5  the average metrics.calinski_harabaz_score is : 5.69772301874
For n_clusters = 6 The average silhouette_score is : 0.185685543781
For n_clusters = 6  the average metrics.calinski_harabaz_score is : 5.49043933528
For n_clusters = 7 The average silhouette_score is : 0.257798782989
For n_clusters = 7  the average metrics.calinski_harabaz_score is : 5.95852242264
For n_clusters = 8 The average silhouette_score is : 0.295845869636
For n_clusters = 8  the average metr

In [153]:
n_clusters = 4
pca = PCA().fit(data)
cluster_labels = kmeans.fit_predict(reduced_data[:,:2])
plt.figure()
plt.plot(range(len(pca.explained_variance_ratio_)), np.cumsum(pca.explained_variance_ratio_))
plt.axvline(pca_comps, color="red")  
plt.ylim(0.0,1.1)
plt.show()

In [166]:
reduced_data = pca.transform(data)
#print ("Reduced data: ",reduced_data.shape)
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10)
kmeans.fit(reduced_data[:,:2])

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min()-.05, reduced_data[:, 0].max()+.05
y_min, y_max = reduced_data[:, 1].min()-.05, reduced_data[:, 1].max()+.05
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(10,10))
#plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap= plt.cm.Pastel2,#cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

#plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=4)

#listA=df[df['readmitted'] == 0].index
#plt.plot(reduced_data[listA, 0], reduced_data[listA, 1],'k.', markersize=4, c='g', label='No Readmitted')
#listB=df[df['readmitted'] == 29].index
#plt.plot(reduced_data[listB, 0],reduced_data[listB, 1],'k.', markersize=4, c='r',label ='<30')
#listC=df[df['readmitted'] == 31].index
#plt.plot(reduced_data[listC, 0],reduced_data[listC, 1],'k.', markersize=4, c='b',label ='>30')

listA=df_3[df_3['Country_Spain'] == 1].index
plt.plot(reduced_data[listA, 0], reduced_data[listA, 1],'k.', markersize=4, c='r', label='Spain')
listB=df_3[df_3['Country_Spain'] == 0].index
plt.plot(reduced_data[listB, 0],reduced_data[listB, 1],'k.', markersize=4, c='g',label ='Not Spain')

#Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
#       'Chronic_pulmonary_disease', 'Myocardial_infarction',
#       'Congestive_heart_failure'

# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[0, 0], centroids[0, 1],
            marker='>', s=169, linewidths=3, 
            color='black', zorder=10)
plt.scatter(centroids[1, 0], centroids[1, 1],
            marker='H', s=169, linewidths=3, 
            color='black', zorder=10)
plt.scatter(centroids[2, 0], centroids[2, 1],
            marker='>', s=169, linewidths=3, 
            color='blue', zorder=10)
plt.scatter(centroids[3, 0], centroids[3, 1],
            marker='H', s=169, linewidths=3, 
            color='blue', zorder=10)
plt.title('K-means clustering on the rare diseases dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.legend()
plt.show()

In [161]:
n_clusters=4
num_age = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    #print(df_4[i][df_3.columns.get_loc("Age")])
    #print(num_age[cluster_labels[i]])
    num_age[cluster_labels[i]] = num_age[cluster_labels[i]] + df_4[i][df_3.columns.get_loc("Age")]
mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
print ("average Age = ", list(map(truediv,num_age,mida_cluster)))

num_Myocardial_infarction = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Country_Spain")] == 1):
        num_Myocardial_infarction[cluster_labels[i]] = num_Myocardial_infarction[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("country_Spain = ", num_Myocardial_infarction)

num_Myocardial_infarction = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Country_Iran")] == 1):
        num_Myocardial_infarction[cluster_labels[i]] = num_Myocardial_infarction[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("country_Iran = ", num_Myocardial_infarction)

num_Myocardial_infarction = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Country_France")] == 1):
        num_Myocardial_infarction[cluster_labels[i]] = num_Myocardial_infarction[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("country_France = ", num_Myocardial_infarction)

average Age =  [26.600000000000001, 15.285714285714286, 23.0, 23.0]
cluster_sizes =  [15, 7, 1, 2]
country_Spain =  [15, 0, 0, 2]
cluster_sizes =  [15, 7, 1, 2]
country_Iran =  [0, 3, 0, 0]
cluster_sizes =  [15, 7, 1, 2]
country_France =  [0, 1, 0, 0]


In [163]:
#P-values of attributes of each cluster
import scipy.stats as stats
#true_mu = 0
listaA=df_3.columns.tolist()
cluster1=df_4[kmeans.labels_==0]
cluster2=df_4[kmeans.labels_==1]
cluster_labels=kmeans.labels_
#print(cluster1.shape)
#print(cluster2.shape)
contador=0
for z in range(0,len(df_3.columns.tolist())):
    two_sample = stats.ttest_ind(cluster1[:,z],cluster2[:,z])
    if(two_sample[1]<0.05):contador=contador+1

w, h = 3, contador
Matrix = [[0.0 for x in range(w)] for y in range(h)] 
i=0
for z in range(0,len(df_3.columns.tolist())):
    #print(df_2.loc[1,z])
    #print(np.std(cluster1[:,z]))
    #print(df_2.values[:,z])
    two_sample = stats.ttest_ind(cluster1[:,z],cluster2[:,z])
    #two_sample = stats.chisquare(cluster1[:,8],cluster2[:,8])
    if(two_sample[1]<0.05): 
        #print(i)
        Matrix[i][0]=listaA[z]
        Matrix[i][1]='{0:.400f}'.format(two_sample[1])
        Matrix[i][2]=two_sample[1]
        i=i+1
arr = np.array(Matrix)
arr = arr[arr[:,1].argsort()]
#print(arr)

for z in range(0,h):
    if (0<1):
        num_hipertensos = [0] * n_clusters
        for i in range(len(data[:,:2])):
            num_hipertensos[cluster_labels[i]] = num_hipertensos[cluster_labels[i]] + df_4[i][df_3.columns.get_loc(arr[z,0])]
        mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
        print ((arr[z,0]),": ", list(map(truediv,num_hipertensos,mida_cluster)),", p-value: ",arr[z,2])
    else:
        num_hipertensos = [0] * n_clusters
        for i in range(len(data[:,:2])):
            #print("z",z)
            #print("i",i)
            if (data[i][df_3.columns.get_loc(arr[z,0])] == 1):
                num_hipertensos[cluster_labels[i]] = num_hipertensos[cluster_labels[i]] + 1
        mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
        percentage=list(map(truediv,num_hipertensos,mida_cluster))
        percentage[0]=round(percentage[0],2)
        percentage[1]=round(percentage[1],2)
        print ((arr[z,0]),": ", num_hipertensos,", percentage: ",percentage, "p-value: ",arr[z,2])

Country_Spain :  [1.0, 0.0, 1.0, 0.0] , p-value:  0.0
Country_Iran :  [0.0, 0.42857142857142855, 0.0, 0.0] , p-value:  0.00451667389589
Age :  [26.600000000000001, 15.285714285714286, 23.0, 23.0] , p-value:  0.0393183110534
